Coefficient of variation, add warm-up for training the mean, implement $\beta$-NLL loss function during training, 10-fold cross validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,root_mean_squared_error

import torch
import torch.nn as nn   
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
import torch.nn.functional as F

import optuna
import random

import pickle

In [ ]:
training_cells = pd.read_csv("../Data_preprocessing/training.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
test_in_cells = pd.read_csv("../Data_preprocessing/test_in.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()
test_out_cells = pd.read_csv("../Data_preprocessing/test_out.csv",header=None).to_numpy(dtype=str).reshape(-1,).tolist()

num_training_cells = len(training_cells)
num_test_in_cells = len(test_in_cells)
num_test_out_cells = len(test_out_cells)

a = np.loadtxt('../Empirical_model_fitting/Empirical_parameters_global_train_py.csv').item()

In [ ]:
X_train_PCA = np.loadtxt("Processed_input_output/X_train_PCA.csv",delimiter=",")
X_test_in_PCA = np.loadtxt("Processed_input_output/X_test_in_PCA.csv",delimiter=",")
X_test_out_PCA = np.loadtxt("Processed_input_output/X_test_out_PCA.csv",delimiter=",")

N_train = np.loadtxt("Processed_input_output/N_train.csv",delimiter=",")
N_test_in = np.loadtxt("Processed_input_output/N_test_in.csv",delimiter=",")
N_test_out = np.loadtxt("Processed_input_output/N_test_out.csv",delimiter=",")

Q_train = np.array([np.linspace(1,0.8,21)] * num_training_cells)
Q_test_in = np.array([np.linspace(1,0.8,21)] * num_test_in_cells)
Q_test_out = np.array([np.linspace(1,0.8,21)] * num_test_out_cells)

Load predefined 10-fold cross validation

In [4]:
with open('training_cells_CV.pkl','rb') as f:
    training_cells_CV = pickle.load(f)

with open('val_cells_CV.pkl','rb') as f:
    val_cells_CV = pickle.load(f)

train_dataset_CV = []
val_dataset_CV = []
for fold in range(10):
    train_cells = training_cells_CV[fold]
    val_cells = val_cells_CV[fold]

    X_train_fold = X_train_PCA[train_cells]
    X_val_fold = X_train_PCA[val_cells]
   
    Y_train_fold = N_train[train_cells]
    Y_val_fold = N_train[val_cells]

    X_train_fold = torch.tensor(X_train_fold,dtype=torch.float32)
    X_val_fold = torch.tensor(X_val_fold,dtype=torch.float32)

    Y_train_fold = torch.tensor(Y_train_fold,dtype=torch.float32)
    Y_val_fold = torch.tensor(Y_val_fold,dtype=torch.float32)


    train_dataset = TensorDataset(X_train_fold,Y_train_fold)
    val_dataset = TensorDataset(X_val_fold,Y_val_fold)

    train_dataset_CV.append(train_dataset)
    val_dataset_CV.append(val_dataset)

Define end-to-end objective function and empirical models for evaluation

In [5]:
def empirical_model(global_p,b1,b2,b3,N_eq,b_weight=[1e-3,1e2,1e2]):
    a1 = global_p
    # Match the scaling for end-to-end formulation
    return 1 - b1*N_eq**a1*b_weight[0] - 1/(1+np.exp((b2*b_weight[1]-N_eq)/(b3*b_weight[2])))


def empirical_model_seperate(global_p,b1,b2,b3,N_eq,b_weight=[1e-3,1e2,1e2]):
    a1 = global_p
    # Match the scaling for end-to-end formulation
    return [1 - b1*N_eq**a1*b_weight[0] - 1/(1+np.exp((b2*b_weight[1]-N_eq)/(b3*b_weight[2]))),
            - b1*N_eq**a1*b_weight[0],
            - 1/(1+np.exp((b2*b_weight[1]-N_eq)/(b3*b_weight[2])))]

Type 1: Predict the coefficient of variation of the curve

In [6]:
# Define the NN model function
class Network_cof_variation(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(Network_cof_variation, self).__init__()
        # Define the layers
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_sizes[0]))  # Input layer
        for i in range(1, len(hidden_sizes)):
            self.layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))  # Hidden layers
        self.mean_layer = nn.Linear(hidden_sizes[-1], output_size - 1)  # Output for b1, b2, b3
        self.coe_var_layer = nn.Linear(hidden_sizes[-1], 1)  # Output for coefficient of variance
        self._initialize_weights()
        
    def forward(self, x):
        # Pass through hidden layers with ReLU activations
        for layer in self.layers:
            x = torch.relu(layer(x))
        
        # Separate outputs for means (b1, b2, b3) and coefficient of variance
        # means = torch.sigmoid(self.mean_layer(x)) * (40 - 1) + 1  # Outputs in range [1, 40]
        means = self.mean_layer(x)
        # means = torch.relu(means)
        means = torch.sigmoid(means) * (40 - 1) + 1  # Outputs in range [1, 30]

        coe_var = F.softplus(self.coe_var_layer(x))  # Scale to [0, inf] for CV

        return torch.cat((means, coe_var), dim=1)  # Concatenate means and coe_var
    
    def _initialize_weights(self):
        # He Normal initialization for weights, zero for biases
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                torch.nn.init.kaiming_normal_(layer.weight, nonlinearity='relu')
                if layer.bias is not None:
                    torch.nn.init.zeros_(layer.bias)
                    
        # Xavier Normal initialization for mean_layer (sigmoid activation)
        torch.nn.init.xavier_normal_(self.mean_layer.weight)
        # torch.nn.init.kaiming_normal_(self.mean_layer.weight, nonlinearity='relu')
        torch.nn.init.zeros_(self.mean_layer.bias)
        
        # Xavier Normal initialization for coe_var_layer (Softplus activation)
        # torch.nn.init.xavier_normal_(self.coe_var_layer.weight)
        torch.nn.init.ones_(self.coe_var_layer.bias)
        torch.nn.init.zeros_(self.coe_var_layer.bias)


# Define the early stopping class
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [7]:
# Define the loss functions
# regular NLL loss for tracking traning performance
# Beta-NLL loss for backpropagation

def end_to_end_loss_NN_NLL_cov(output, target,a=0.5,warmup=True,b_weight=[1e-3,1e2,1e2]):
    # batch size and number of measurements
    (batch_size, len_interp) = target.size()

    # Predicted means (b1, b2, b3) and coefficient of variation
    b1 = output[:, 0].unsqueeze(1)  # b1 mean prediction
    b2 = output[:, 1].unsqueeze(1)  # b2 mean prediction
    b3 = output[:, 2].unsqueeze(1)  # b3 mean prediction
    # log_coe_var = output[:, 3].unsqueeze(1)  # log of coefficient of variation for all predictions
    coe_var = output[:, 3].unsqueeze(1)  # coefficient of variation for all predictions
    
    # Define ones
    I_nm = torch.ones_like(target, dtype=torch.float32)
    I_1m = torch.ones((1, len_interp), dtype=torch.float32)
    
    # Define Q (true capacity trajectories)
    Q = torch.linspace(1, 0.8, len_interp).unsqueeze(0).repeat(batch_size, 1) * 100 # in percentage scale
    
    # Calculate term_2
    term_2 = b1 * I_1m * b_weight[0] * torch.pow(target, a)
    
    # Calculate term_3
    exp_input = ((b2 * I_1m * b_weight[1]) - target) / ((b3 * I_1m * b_weight[2]) + 1e-6)
    # exp_input_clamped = 80 * torch.tanh(exp_input / 80)  # Smooth clipping using tanh to avoid overflow/underflow
    exp_input_clamped = torch.clamp(exp_input, min=-80, max=80)  # Clamp to avoid overflow/underflow
    term_3 = I_nm / (I_nm + torch.exp(exp_input_clamped))
    
    # Calculate predicted values based on term_2 and term_3
    predicted_values = (I_nm  -  term_2 - term_3) * 100
    # Calculate the variance and log-variance
    std = (100 - predicted_values) * coe_var # standard deviation based on coefficient of variation. As capacity fade increases, variance increases
    var = torch.pow(std, 2)
    var = torch.clamp(var, min=1e-6)  # Clamp variance to avoid division by zero and log of zero
    log_var = torch.log(var) 
       
    if warmup:
        log_var = 0
        var = 1

    # Negative log-likelihood loss based on Gaussian distribution        
    nll_loss = 0.5 * torch.mean((Q - predicted_values) ** 2 / var + log_var)
    
    return nll_loss


def end_to_end_loss_NN_betaNLL(output, target,beta=0.5,a=0.5,warmup=True,b_weight=[1e-3,1e2,1e2]):
    # batch size and number of measurements
    (batch_size, len_interp) = target.size()

    # Predicted means (b1, b2, b3) and coefficient of variation
    b1 = output[:, 0].unsqueeze(1)  # b1 mean prediction
    b2 = output[:, 1].unsqueeze(1)  # b2 mean prediction
    b3 = output[:, 2].unsqueeze(1)  # b3 mean prediction
    # log_coe_var = output[:, 3].unsqueeze(1)  # log of coefficient of variation for all predictions
    coe_var = output[:, 3].unsqueeze(1)  # coefficient of variation for all predictions
    
    # Define ones
    I_nm = torch.ones_like(target, dtype=torch.float32)
    I_1m = torch.ones((1, len_interp), dtype=torch.float32)
    
    # Define Q (true capacity trajectories)
    Q = torch.linspace(1, 0.8, len_interp).unsqueeze(0).repeat(batch_size, 1) * 100 # in percentage scale
    
    # Calculate term_2
    term_2 = b1 * I_1m * b_weight[0] * torch.pow(target, a)
    
    # Calculate term_3
    exp_input = ((b2 * I_1m * b_weight[1]) - target) / ((b3 * I_1m * b_weight[2]) + 1e-6)
    # exp_input_clamped = 80 * torch.tanh(exp_input / 80)  # Smooth clipping using tanh to avoid overflow/underflow
    exp_input_clamped = torch.clamp(exp_input, min=-80, max=80)  # Clamp to avoid overflow/underflow
    term_3 = I_nm / (I_nm + torch.exp(exp_input_clamped))
    
    # Calculate predicted values based on term_2 and term_3
    predicted_values = (I_nm  -  term_2 - term_3) * 100
    # Calculate the variance and log-variance
    std = (100 - predicted_values) * coe_var # standard deviation based on coefficient of variation. As capacity fade increases, variance increases
    var = torch.pow(std, 2)
    var = torch.clamp(var, min=1e-6)  # Clamp variance to avoid division by zero and log of zero
    log_var = torch.log(var) 
       
    if warmup or beta==0:
        log_var = 0
        var = 1
        # Negative log-likelihood loss based on Gaussian distribution        
        nll_loss = 0.5 * torch.mean((Q - predicted_values) ** 2 / var + log_var)
    else:
        nll_loss = torch.mean((0.5 * ((Q - predicted_values) ** 2 / var + log_var)) * var.detach() ** beta)
    
    return nll_loss

# Hyperparameter optimization with 10-fold CV

Set up neural network functions

In [8]:
seed_ = 42
random.seed(seed_)
torch.manual_seed(seed_)
np.random.seed(seed_)
torch.use_deterministic_algorithms(True)

Process data for training NN (create a validation subset for early stopping and model selection)

In [ ]:
best_val_loss = np.inf
best_model_path = "Best_network/best_NN_NLL_cov_V2.pth"
b_weight = [5e-4,30,5] # Scaling weights for b1, b2, b3
def objective(trial):
    global best_val_loss

    val_loss_CV = []
    model_state_dict = {}

    # Define the input and output sizes
    input_size = 10  # 10 PCA features
    output_size = 4  # 3 empirical model parameters + 1 variance related value
    beta = 0.5 # Beta value for Beta-NLL loss
    max_grad_norm = 1.0

    # Define the hyperparameters to tune
    num_layer = trial.suggest_int('num_layer', 2, 5)
    num_neuron = trial.suggest_int('num_neuron', 5, 10)
    hidden_sizes = [num_neuron] * num_layer
    batch_size = trial.suggest_int('batch_size', 32, 45)
    lr = trial.suggest_float('lr', 5e-4, 5e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-6, log=True)
    warmup_epochs = trial.suggest_int('warmup_epochs', 500, 800)

    for fold in range(10):
        train_dataset = train_dataset_CV[fold]
        val_dataset = val_dataset_CV[fold]
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
        # Create the model
        model = Network_cof_variation(input_size, hidden_sizes, output_size)
        criterion = end_to_end_loss_NN_betaNLL
        criterion_val = end_to_end_loss_NN_NLL_cov
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        warmup = True # Define this for loss function to use warmup or not
        # Freeze weights for coefficient of variation layer before warm-up epochs
        for param in model.coe_var_layer.parameters():
            param.requires_grad = False

        
        # Training loop
        num_epochs = 3000
        warmup_epochs_early_stopping = warmup_epochs + 100  # Early stopping warm-up epochs
        early_stopping = EarlyStopping(patience=10, min_delta=1e-4)
        
        for epoch in range(num_epochs):
            if epoch == warmup_epochs:
                warmup = False
                for param in model.coe_var_layer.parameters():
                    param.requires_grad = True
            # Training phase
            model.train()
            train_loss = 0.0
            for batch in train_loader:
                inputs, labels = batch
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels, beta, a, warmup, b_weight)
                loss_val = criterion_val(outputs, labels, a, warmup, b_weight)

                loss.backward()
                if epoch >= warmup_epochs:
                    # Gradient clipping
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                train_loss += loss_val.item()
            train_loss /= len(train_loader)
            
            # Validation phase
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    loss = criterion_val(outputs, targets, a, warmup, b_weight)
                    val_loss += loss.item()
            val_loss /= len(val_loader)
            
            # Check early stopping condition after warmup
            if epoch > warmup_epochs_early_stopping:
                early_stopping(val_loss)
                if early_stopping.early_stop:
                    print(f"Early stopping at epoch {epoch + 1}")
                    break

        val_loss_CV.append(val_loss)
        model_state_dict[fold] = model.state_dict()
    
    val_loss_overall = np.mean(val_loss_CV)
    print(f"Validation loss: {val_loss_CV}")
    # Check if the current validation loss is the best
    if val_loss_overall < best_val_loss:
        best_val_loss = val_loss_overall
        # Save the best model
        torch.save({'model_state':model_state_dict,
                    'lr':lr,
                    'num_neuron':num_neuron,
                    'num_layer':num_layer,
                    'batch_size':batch_size,
                    'weight_decay':weight_decay,
                    'warmup_epochs':warmup_epochs}, best_model_path)

    return val_loss_overall

# Create an Optuna study
sampler = optuna.samplers.TPESampler(seed=seed_)  # Make the sampler deterministic
study = optuna.create_study(sampler=sampler, direction='minimize')

# Run the optimization
study.optimize(objective, n_trials=50)

# Print the best parameters
print("Best hyperparameters:", study.best_params)

[I 2024-10-30 21:33:38,757] A new study created in memory with name: no-name-1454c813-df69-420a-9050-b5f43ff5f674


Early stopping at epoch 662
Early stopping at epoch 665
Early stopping at epoch 663
Early stopping at epoch 658
Early stopping at epoch 673
Early stopping at epoch 671
Early stopping at epoch 658
Early stopping at epoch 660
Early stopping at epoch 658


[I 2024-10-30 21:33:56,861] Trial 0 finished with value: 5.872484445571899 and parameters: {'num_layer': 3, 'num_neuron': 10, 'batch_size': 42, 'lr': 0.007875660249889864, 'weight_decay': 2.0513382630874456e-08, 'warmup_epochs': 546}. Best is trial 0 with value: 5.872484445571899.


Early stopping at epoch 664
Validation loss: [4.984686374664307, 18.311635971069336, 2.1316568851470947, 5.804723262786865, 3.1161296367645264, 3.59780216217041, 11.753974914550781, 3.308239221572876, 2.4160423278808594, 3.2999536991119385]
Early stopping at epoch 924
Early stopping at epoch 920
Early stopping at epoch 905
Early stopping at epoch 910
Early stopping at epoch 917
Early stopping at epoch 912
Early stopping at epoch 903
Early stopping at epoch 911
Early stopping at epoch 915


[I 2024-10-30 21:34:23,392] Trial 1 finished with value: 626095.0360554695 and parameters: {'num_layer': 2, 'num_neuron': 10, 'batch_size': 40, 'lr': 0.013035123791853833, 'weight_decay': 1.0994335574766189e-08, 'warmup_epochs': 791}. Best is trial 0 with value: 5.872484445571899.


...

Early stopping at epoch 765
Validation loss: [2.55228328704834, 6138947.5, 2.5385477542877197, 1.903812050819397, 1.727954626083374, 3.548551559448242, 1.4167394638061523, 1.9315441846847534, 1.5205374956130981, 1.8670755624771118]
Early stopping at epoch 771
Early stopping at epoch 783
Early stopping at epoch 771
Early stopping at epoch 773
Early stopping at epoch 771
Early stopping at epoch 771
Early stopping at epoch 771
Early stopping at epoch 771
Early stopping at epoch 775


[I 2024-10-30 21:53:03,484] Trial 48 finished with value: 75912.64843539 and parameters: {'num_layer': 5, 'num_neuron': 5, 'batch_size': 40, 'lr': 0.004789444071084909, 'weight_decay': 7.282281486411192e-07, 'warmup_epochs': 659}. Best is trial 20 with value: 1.6530466556549073.


Early stopping at epoch 777
Validation loss: [3.019744634628296, 759094.5, 1.223127841949463, 3.4810173511505127, 2.444221258163452, 9.436936378479004, 7.002040386199951, 1.4709240198135376, 2.1856231689453125, 1.7207188606262207]
Early stopping at epoch 705
Early stopping at epoch 705
Early stopping at epoch 705
Early stopping at epoch 724
Early stopping at epoch 1125
Early stopping at epoch 705
Early stopping at epoch 705
Early stopping at epoch 927
Early stopping at epoch 770


[I 2024-10-30 21:53:22,869] Trial 49 finished with value: 1.774815607070923 and parameters: {'num_layer': 4, 'num_neuron': 6, 'batch_size': 43, 'lr': 0.0009027238512416587, 'weight_decay': 1.861176803145822e-08, 'warmup_epochs': 593}. Best is trial 20 with value: 1.6530466556549073.


Early stopping at epoch 790
Validation loss: [2.1141748428344727, 2.304706573486328, 1.4957414865493774, 2.028783082962036, 1.3187745809555054, 2.9198496341705322, 1.237215280532837, 1.5420382022857666, 1.3971918821334839, 1.3896805047988892]
Best hyperparameters: {'num_layer': 4, 'num_neuron': 5, 'batch_size': 43, 'lr': 0.0010279980160927279, 'weight_decay': 3.5722029462803684e-07, 'warmup_epochs': 667}


# Case 1: Single model for each fold

In [ ]:
best_model_dict = torch.load(best_model_path)
hidden_sizes = [best_model_dict['num_neuron']]*best_model_dict['num_layer']

input_size = 10
output_size = 4
# Test features as tensor
X_train_PCA_tensor = torch.tensor(X_train_PCA,dtype = torch.float32)
X_test_in_PCA_tensor = torch.tensor(X_test_in_PCA,dtype=torch.float32)
X_test_out_PCA_tensor = torch.tensor(X_test_out_PCA,dtype=torch.float32)

b_train_pred = []
b_test_in_pred = []
b_test_out_pred = []

Q_train_pred = []
Q_test_in_pred = []
Q_test_out_pred = []

cov_train_pred = []
cov_test_in_pred = []
cov_test_out_pred = []

for fold in range(10):
    # Load the model for the current fold
    model = Network_cof_variation(input_size,hidden_sizes,output_size)
    model.load_state_dict(best_model_dict['model_state'][fold])

    # Evaluate the model with the completed training set and test sets
    model.eval()
    with torch.no_grad():
        b_train_pred_fold = model(X_train_PCA_tensor).numpy()
        b_test_in_pred_fold = model(X_test_in_PCA_tensor).numpy()
        b_test_out_pred_fold = model(X_test_out_PCA_tensor).numpy()

    Q_train_pred_fold = []
    cov_train_pred_fold = []
    for i in range(num_training_cells):
        b = b_train_pred_fold[i,:3]
        cov = b_train_pred_fold[i,3]
        Q_train_pred_fold.append(np.clip(empirical_model(a,*b,N_train[i],b_weight),0,1))
        cov_train_pred_fold.append(cov)
    
    Q_test_in_pred_fold = []
    cov_test_in_pred_fold = []
    for i in range(num_test_in_cells):
        b = b_test_in_pred_fold[i,:3]
        cov = b_test_in_pred_fold[i,3]
        Q_test_in_pred_fold.append(np.clip(empirical_model(a,*b,N_test_in[i],b_weight),0,1))
        cov_test_in_pred_fold.append(cov)

    Q_test_out_pred_fold = []
    cov_test_out_pred_fold = []
    for i in range(num_test_out_cells):
        b = b_test_out_pred_fold[i,:3]
        cov = b_test_out_pred_fold[i,3]
        Q_test_out_pred_fold.append(np.clip(empirical_model(a,*b,N_test_out[i],b_weight),0,1))
        cov_test_out_pred_fold.append(cov)

    # Save the results from the current fold
    b_train_pred.append(np.array(b_train_pred_fold))
    b_test_in_pred.append(np.array(b_test_in_pred_fold))
    b_test_out_pred.append(np.array(b_test_out_pred_fold))

    Q_train_pred.append(np.array(Q_train_pred_fold))
    Q_test_in_pred.append(np.array(Q_test_in_pred_fold))
    Q_test_out_pred.append(np.array(Q_test_out_pred_fold))

    cov_train_pred.append(np.array(cov_train_pred_fold))
    cov_test_in_pred.append(np.array(cov_test_in_pred_fold))
    cov_test_out_pred.append(np.array(cov_test_out_pred_fold))


# Write the results to files
with open('Empirical_parameter_results/E2E_NNE_V2/b_train_pred_1.pkl','wb') as f:
    pickle.dump(b_train_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/b_test_in_pred_1.pkl','wb') as f:
    pickle.dump(b_test_in_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/b_test_out_pred_1.pkl','wb') as f:
    pickle.dump(b_test_out_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_train_pred_1.pkl','wb') as f:
    pickle.dump(Q_train_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_in_pred_1.pkl','wb') as f:
    pickle.dump(Q_test_in_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_out_pred_1.pkl','wb') as f:
    pickle.dump(Q_test_out_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/cov_train_pred_1.pkl','wb') as f:
    pickle.dump(cov_train_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/cov_test_in_pred_1.pkl','wb') as f:
    pickle.dump(cov_test_in_pred,f)

with open('Empirical_parameter_results/E2E_NNE_V2/cov_test_out_pred_1.pkl','wb') as f:
    pickle.dump(cov_test_out_pred,f)

In [ ]:
MAE_training = []
MAE_test_in = []
MAE_test_out = []

RMSE_training = []
RMSE_test_in = []
RMSE_test_out = []

for fold in range(10):
    MAE_training_fold = []
    MAE_test_in_fold = []
    MAE_test_out_fold = []

    RMSE_training_fold = []
    RMSE_test_in_fold = []
    RMSE_test_out_fold = []

    for i in range(num_training_cells):
        MAE_training_fold.append(mean_absolute_error(Q_train[i],Q_train_pred[fold][i]))
        RMSE_training_fold.append(root_mean_squared_error(Q_train[i],Q_train_pred[fold][i]))

    for i in range(num_test_in_cells):
        MAE_test_in_fold.append(mean_absolute_error(Q_test_in[i],Q_test_in_pred[fold][i]))
        RMSE_test_in_fold.append(root_mean_squared_error(Q_test_in[i],Q_test_in_pred[fold][i]))

    for i in range(num_test_out_cells):
        MAE_test_out_fold.append(mean_absolute_error(Q_test_out[i],Q_test_out_pred[fold][i]))
        RMSE_test_out_fold.append(root_mean_squared_error(Q_test_out[i],Q_test_out_pred[fold][i]))

    MAE_training.append(np.mean(MAE_training_fold)*100)
    MAE_test_in.append(np.mean(MAE_test_in_fold)*100)
    MAE_test_out.append(np.mean(MAE_test_out_fold)*100)

    RMSE_training.append(np.mean(RMSE_training_fold)*100)
    RMSE_test_in.append(np.mean(RMSE_test_in_fold)*100)
    RMSE_test_out.append(np.mean(RMSE_test_out_fold)*100)

result_dict = {'MAE_training':MAE_training,
                'MAE_test_in':MAE_test_in,
                'MAE_test_out':MAE_test_out,
                'RMSE_training':RMSE_training,
                'RMSE_test_in':RMSE_test_in,
                'RMSE_test_out':RMSE_test_out}

with open('Empirical_parameter_results/E2E_NNE_V2/result_dict.pkl','wb') as f:
    pickle.dump(result_dict,f)

mean_MAE_training = np.mean(MAE_training)
mean_MAE_test_in = np.mean(MAE_test_in)
mean_MAE_test_out = np.mean(MAE_test_out)

mean_RMSE_training = np.mean(RMSE_training)
mean_RMSE_test_in = np.mean(RMSE_test_in)
mean_RMSE_test_out = np.mean(RMSE_test_out)

print(f"Mean MAE training: {mean_MAE_training:.2f}")
print(f"Mean MAE test in: {mean_MAE_test_in:.2f}")
print(f"Mean MAE test out: {mean_MAE_test_out:.2f}")

print(f"Mean RMSE training: {mean_RMSE_training:.2f}")
print(f"Mean RMSE test in: {mean_RMSE_test_in:.2f}")
print(f"Mean RMSE test out: {mean_RMSE_test_out:.2f}")

Mean MAE training: 1.85
Mean MAE test in: 2.43
Mean MAE test out: 7.84
Mean RMSE training: 2.24
Mean RMSE test in: 2.97
Mean RMSE test out: 9.96


In [13]:
print(f"MAE training: {MAE_training}")
print(f"MAE test in: {MAE_test_in}")
print(f"MAE test out: {MAE_test_out}")

MAE training: [1.643545350505334, 2.09967585249802, 1.7676023752392962, 1.974062259304656, 1.8837080791359464, 1.8206665489696234, 1.7522265953285927, 1.8638849341687598, 1.9714080606586553, 1.7470637235898319]
MAE test in: [2.3389947311835124, 2.4640858057345683, 2.363188725121361, 2.564984971141379, 3.4211871588080895, 2.102207843626132, 2.2165959025676902, 1.986713193687822, 2.4934434410148967, 2.3846918264725514]
MAE test out: [12.384958694318204, 7.712989836741765, 4.418280810455669, 13.134221757590508, 6.314898340426757, 3.045295932947437, 4.014358531770969, 5.434284287702664, 11.997625725366785, 9.924846708968296]


# Case 2: Ensemble of 5 models for each fold

In [ ]:
# Load the hyperparameters for the best model
hidden_sizes = [best_model_dict['num_neuron']]*best_model_dict['num_layer']
lr = best_model_dict['lr']
batch_size = best_model_dict['batch_size']
weight_decay = best_model_dict['weight_decay']
warmup_epochs = best_model_dict['warmup_epochs']
# Define the warm-up epochs for early stopping
warmup_epochs_early_stopping = warmup_epochs + 100
num_epochs = 3000
# Number of models to train for ensemble
M = 5
# Beta value for Beta-NLL loss
beta = 0.5
# Gradient clipping value
max_grad_norm = 1.0

model_state_dict_5 = {}

for fold in range(10):
    model_state_dict_5[fold] = []
    val_loss_fold = []
    for m in range(M):
        model = Network_cof_variation(input_size, hidden_sizes, output_size)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = end_to_end_loss_NN_betaNLL
        criterion_val = end_to_end_loss_NN_NLL_cov
        early_stopping = EarlyStopping(patience=10, min_delta=1e-4)
        # Freeze weights for coefficient of variation layer before warm-up epochs
        warmup = True
        for param in model.coe_var_layer.parameters():
            param.requires_grad = False

        train_dataset = train_dataset_CV[fold]
        val_dataset = val_dataset_CV[fold]

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

        for epoch in range(num_epochs):
            if epoch == warmup_epochs:
                warmup = False
                for param in model.coe_var_layer.parameters():
                    param.requires_grad = True

            model.train()
            train_loss = 0.0
            for batch in train_loader:
                inputs, labels = batch
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels, beta, a, warmup, b_weight)
                loss_val = criterion_val(outputs, labels, a, warmup, b_weight)
                loss.backward()
                if epoch >= warmup_epochs:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                train_loss += loss_val.item()
            train_loss /= len(train_loader)

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    loss = criterion_val(outputs, targets, a, warmup, b_weight)
                    val_loss += loss.item()
            val_loss /= len(val_loader)

            if epoch > warmup_epochs_early_stopping:
                early_stopping(val_loss)
                if early_stopping.early_stop:
                    print(f"Early stopping at epoch {epoch + 1}")
                    break
        
        model_state_dict_5[fold].append(model.state_dict())
        val_loss_fold.append(val_loss)
        print(f"Fold {fold + 1}, Model {m + 1}, Validation loss: {val_loss}")
    
with open('Best_network/best_NN_NLL_cov_V2_5.pth','wb') as f:
    torch.save({'model_state':model_state_dict_5,
                'lr':lr,
                'num_neuron':best_model_dict['num_neuron'],
                'num_layer':best_model_dict['num_layer'],
                'batch_size':batch_size,
                'weight_decay':weight_decay,
                'warmup_epochs':warmup_epochs},f)

        
        

Early stopping at epoch 779
Fold 1, Model 1, Validation loss: 3.8215935230255127
Early stopping at epoch 779
Fold 1, Model 2, Validation loss: 2.4502148628234863
Early stopping at epoch 779
Fold 1, Model 3, Validation loss: 3.09334397315979
Early stopping at epoch 779
Fold 1, Model 4, Validation loss: 3.1151938438415527
Early stopping at epoch 779
Fold 1, Model 5, Validation loss: 2.4279656410217285
Early stopping at epoch 779
Fold 2, Model 1, Validation loss: 2.0148041248321533
Early stopping at epoch 779
Fold 2, Model 2, Validation loss: 3.1719396114349365
Early stopping at epoch 779
Fold 2, Model 3, Validation loss: 1.9256962537765503
Early stopping at epoch 852
Fold 2, Model 4, Validation loss: 3.7551651000976562
Early stopping at epoch 779
Fold 2, Model 5, Validation loss: 12.877677917480469
Early stopping at epoch 779
Fold 3, Model 1, Validation loss: 1.2635449171066284
Early stopping at epoch 779
Fold 3, Model 2, Validation loss: 1.644436001777649
Early stopping at epoch 779
Fol

In [29]:
b_train_pred_5_all = {}
b_test_in_pred_5_all = {}
b_test_out_pred_5_all = {}

for fold in range(10):
    b_train_pred_5 = []
    b_test_in_pred_5 = []
    b_test_out_pred_5 = []

    for m in range(M):
        model = Network_cof_variation(input_size, hidden_sizes, output_size)
        model.load_state_dict(model_state_dict_5[fold][m])

        model.eval()
        with torch.no_grad():
            b_train_pred_fold_m = model(X_train_PCA_tensor).numpy()
            b_test_in_pred_fold_m = model(X_test_in_PCA_tensor).numpy()
            b_test_out_pred_fold_m = model(X_test_out_PCA_tensor).numpy()

        b_train_pred_5.append(b_train_pred_fold_m)
        b_test_in_pred_5.append(b_test_in_pred_fold_m)
        b_test_out_pred_5.append(b_test_out_pred_fold_m)

    b_train_pred_5_all[fold] = np.array(b_train_pred_5)
    b_test_in_pred_5_all[fold] = np.array(b_test_in_pred_5)
    b_test_out_pred_5_all[fold] = np.array(b_test_out_pred_5)

In [ ]:
with open('Empirical_parameter_results/E2E_NNE_V2/b_train_pred_5_all.pkl','wb') as f:
    pickle.dump(b_train_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/b_test_in_pred_5_all.pkl','wb') as f:
    pickle.dump(b_test_in_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/b_test_out_pred_5_all.pkl','wb') as f:
    pickle.dump(b_test_out_pred_5_all,f)

In [ ]:
Q_train_pred_5_all = {}
Q_test_in_pred_5_all = {}
Q_test_out_pred_5_all = {}

std_train_pred_5_all = {}
std_test_in_pred_5_all = {}
std_test_out_pred_5_all = {}

Q_train_pred_5_ensemble = {}
Q_test_in_pred_5_ensemble = {}
Q_test_out_pred_5_ensemble = {}

std_train_pred_5_ensemble = {}
std_test_in_pred_5_ensemble = {}
std_test_out_pred_5_ensemble = {}

for fold in range(10):
    Q_train_hat_all_models_fold = np.array([[empirical_model(a, b1, b2, b3, N_train[i],b_weight)*100 for i, (b1, b2, b3, _) in enumerate(model_b_train_hat)] for model_b_train_hat in b_train_pred_5_all[fold]])
    Q_test_in_hat_all_models_fold = np.array([[empirical_model(a, b1, b2, b3, N_test_in[i],b_weight)*100 for i, (b1, b2, b3, _) in enumerate(model_b_test_in_hat)] for model_b_test_in_hat in b_test_in_pred_5_all[fold]])
    Q_test_out_hat_all_models_fold = np.array([[empirical_model(a, b1, b2, b3, N_test_out[i],b_weight)*100 for i, (b1, b2, b3, _) in enumerate(model_b_test_out_hat)] for model_b_test_out_hat in b_test_out_pred_5_all[fold]])

    coe_var_train_all_models_fold = np.array([np.repeat(model_b_train_hat[:, 3][:, np.newaxis], Q_train.shape[1], axis=1) for model_b_train_hat in b_train_pred_5_all[fold]])
    coe_var_test_in_all_models_fold = np.array([np.repeat(model_b_test_in_hat[:, 3][:, np.newaxis], Q_test_in.shape[1], axis=1) for model_b_test_in_hat in b_test_in_pred_5_all[fold]])
    coe_var_test_out_all_models_fold = np.array([np.repeat(model_b_test_out_hat[:, 3][:, np.newaxis], Q_test_out.shape[1], axis=1) for model_b_test_out_hat in b_test_out_pred_5_all[fold]])

    # Calculate the variance for each individual model
    std_train_all_models_fold = coe_var_train_all_models_fold * (100 - Q_train_hat_all_models_fold)
    std_test_in_all_models_fold = coe_var_test_in_all_models_fold * (100 - Q_test_in_hat_all_models_fold)
    std_test_out_all_models_fold = coe_var_test_out_all_models_fold * (100 - Q_test_out_hat_all_models_fold)

    var_train_all_models_fold = std_train_all_models_fold ** 2
    var_test_in_all_models_fold = std_test_in_all_models_fold ** 2
    var_test_out_all_models_fold = std_test_out_all_models_fold ** 2

    Q_train_hat_combined_fold = np.mean(Q_train_hat_all_models_fold, axis=0)
    Q_test_in_hat_combined_fold = np.mean(Q_test_in_hat_all_models_fold, axis=0)
    Q_test_out_hat_combined_fold = np.mean(Q_test_out_hat_all_models_fold, axis=0)

    # Calculate the variance of the combined predictions
    var_train_combined_fold = np.mean(var_train_all_models_fold + Q_train_hat_all_models_fold**2 - Q_train_hat_combined_fold**2, axis=0)
    var_test_in_combined_fold = np.mean(var_test_in_all_models_fold + Q_test_in_hat_all_models_fold**2 - Q_test_in_hat_combined_fold**2, axis=0)
    var_test_out_combined_fold = np.mean(var_test_out_all_models_fold + Q_test_out_hat_all_models_fold**2 - Q_test_out_hat_combined_fold**2, axis=0)

    std_train_combined_fold = np.sqrt(var_train_combined_fold)
    std_test_in_combined_fold = np.sqrt(var_test_in_combined_fold)
    std_test_out_combined_fold = np.sqrt(var_test_out_combined_fold)

    Q_train_pred_5_all[fold] = Q_train_hat_all_models_fold
    Q_test_in_pred_5_all[fold] = Q_test_in_hat_all_models_fold
    Q_test_out_pred_5_all[fold] = Q_test_out_hat_all_models_fold

    std_train_pred_5_all[fold] = std_train_all_models_fold
    std_test_in_pred_5_all[fold] = std_test_in_all_models_fold
    std_test_out_pred_5_all[fold] = std_test_out_all_models_fold

    Q_train_pred_5_ensemble[fold] = Q_train_hat_combined_fold
    Q_test_in_pred_5_ensemble[fold] = Q_test_in_hat_combined_fold
    Q_test_out_pred_5_ensemble[fold] = Q_test_out_hat_combined_fold

    std_train_pred_5_ensemble[fold] = std_train_combined_fold
    std_test_in_pred_5_ensemble[fold] = std_test_in_combined_fold
    std_test_out_pred_5_ensemble[fold] = std_test_out_combined_fold

with open('Empirical_parameter_results/E2E_NNE_V2/Q_train_pred_5_all.pkl','wb') as f:
    pickle.dump(Q_train_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_in_pred_5_all.pkl','wb') as f:
    pickle.dump(Q_test_in_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_out_pred_5_all.pkl','wb') as f:
    pickle.dump(Q_test_out_pred_5_all,f)

with open('Empirical-parameter_results/E2E_NNE_V2/std_train_pred_5_all.pkl','wb') as f:
    pickle.dump(std_train_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_in_pred_5_all.pkl','wb') as f:
    pickle.dump(std_test_in_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_out_pred_5_all.pkl','wb') as f:
    pickle.dump(std_test_out_pred_5_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_train_pred_5_ensemble.pkl','wb') as f:
    pickle.dump(Q_train_pred_5_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_in_pred_5_ensemble.pkl','wb') as f:
    pickle.dump(Q_test_in_pred_5_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_out_pred_5_ensemble.pkl','wb') as f:
    pickle.dump(Q_test_out_pred_5_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_train_pred_5_ensemble.pkl','wb') as f:
    pickle.dump(std_train_pred_5_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_in_pred_5_ensemble.pkl','wb') as f:
    pickle.dump(std_test_in_pred_5_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_out_pred_5_ensemble.pkl','wb') as f:
    pickle.dump(std_test_out_pred_5_ensemble,f)


In [ ]:
MAE_training = []
MAE_test_in = []
MAE_test_out = []

RMSE_training = []
RMSE_test_in = []
RMSE_test_out = []

for fold in range(10):
    MAE_training_fold = []
    MAE_test_in_fold = []
    MAE_test_out_fold = []

    RMSE_training_fold = []
    RMSE_test_in_fold = []
    RMSE_test_out_fold = []

    for i in range(num_training_cells):
        MAE_training_fold.append(mean_absolute_error(Q_train[i]*100,Q_train_pred_5_ensemble[fold][i]))
        RMSE_training_fold.append(root_mean_squared_error(Q_train[i]*100,Q_train_pred_5_ensemble[fold][i]))

    for i in range(num_test_in_cells):
        MAE_test_in_fold.append(mean_absolute_error(Q_test_in[i]*100,Q_test_in_pred_5_ensemble[fold][i]))
        RMSE_test_in_fold.append(root_mean_squared_error(Q_test_in[i]*100,Q_test_in_pred_5_ensemble[fold][i]))

    for i in range(num_test_out_cells):
        MAE_test_out_fold.append(mean_absolute_error(Q_test_out[i]*100,Q_test_out_pred_5_ensemble[fold][i]))
        RMSE_test_out_fold.append(root_mean_squared_error(Q_test_out[i]*100,Q_test_out_pred_5_ensemble[fold][i]))

    MAE_training.append(np.mean(MAE_training_fold))
    MAE_test_in.append(np.mean(MAE_test_in_fold))
    MAE_test_out.append(np.mean(MAE_test_out_fold))

    RMSE_training.append(np.mean(RMSE_training_fold))
    RMSE_test_in.append(np.mean(RMSE_test_in_fold))
    RMSE_test_out.append(np.mean(RMSE_test_out_fold))

result_dict = {'MAE_training':MAE_training,
                'MAE_test_in':MAE_test_in,
                'MAE_test_out':MAE_test_out,
                'RMSE_training':RMSE_training,
                'RMSE_test_in':RMSE_test_in,
                'RMSE_test_out':RMSE_test_out}

with open('Empirical_parameter_results/E2E_NNE_V2/result_dict_5.pkl','wb') as f:
    pickle.dump(result_dict,f)

mean_MAE_training = np.mean(MAE_training)
mean_MAE_test_in = np.mean(MAE_test_in)
mean_MAE_test_out = np.mean(MAE_test_out)

mean_RMSE_training = np.mean(RMSE_training)
mean_RMSE_test_in = np.mean(RMSE_test_in)
mean_RMSE_test_out = np.mean(RMSE_test_out)

print(f"Mean MAE training: {mean_MAE_training:.2f}")
print(f"Mean MAE test in: {mean_MAE_test_in:.2f}")
print(f"Mean MAE test out: {mean_MAE_test_out:.2f}")

print(f"Mean RMSE training: {mean_RMSE_training:.2f}")
print(f"Mean RMSE test in: {mean_RMSE_test_in:.2f}")
print(f"Mean RMSE test out: {mean_RMSE_test_out:.2f}")

Mean MAE training: 1.70
Mean MAE test in: 2.00
Mean MAE test out: 6.44
Mean RMSE training: 2.10
Mean RMSE test in: 2.45
Mean RMSE test out: 8.05


In [33]:
print(f"MAE training: {MAE_training}")
print(f"MAE test in: {MAE_test_in}")
print(f"MAE test out: {MAE_test_out}")

MAE training: [1.78373181841315, 1.6457707954245309, 1.6905657949088835, 1.6914115805063543, 1.6920497789455553, 1.7432345106643212, 1.689637178767997, 1.6798989018940038, 1.6583976309139477, 1.7403471134382769]
MAE test in: [2.0343837807806238, 1.9400271014985158, 2.2426159246530264, 2.0262238336288414, 2.0114062041676566, 1.9822969096112968, 2.001414718506352, 2.0507026018673056, 1.9423711434600974, 1.7585559089400231]
MAE test out: [4.658718441482848, 7.166958282987388, 9.293466977805746, 4.6115828530037035, 5.957440617093438, 4.959447638833681, 6.003882856250859, 8.961618578398028, 7.003241850328285, 5.793628848759678]


# Case 3: Ensemble of 10 models for each fold

In [ ]:
# Load the hyperparameters for the best model
hidden_sizes = [best_model_dict['num_neuron']]*best_model_dict['num_layer']
lr = best_model_dict['lr']
batch_size = best_model_dict['batch_size']
weight_decay = best_model_dict['weight_decay']
warmup_epochs = best_model_dict['warmup_epochs']
# Define the warm-up epochs for early stopping
warmup_epochs_early_stopping = warmup_epochs + 100
num_epochs = 3000
# Number of models to train for ensemble
M = 10
# Beta value for Beta-NLL loss
beta = 0.5
# Gradient clipping value
max_grad_norm = 1.0

model_state_dict_10 = {}

for fold in range(10):
    model_state_dict_10[fold] = []
    val_loss_fold = []
    for m in range(M):
        model = Network_cof_variation(input_size, hidden_sizes, output_size)
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = end_to_end_loss_NN_betaNLL
        criterion_val = end_to_end_loss_NN_NLL_cov
        early_stopping = EarlyStopping(patience=10, min_delta=1e-4)
        # Freeze weights for coefficient of variation layer before warm-up epochs
        warmup = True
        for param in model.coe_var_layer.parameters():
            param.requires_grad = False

        train_dataset = train_dataset_CV[fold]
        val_dataset = val_dataset_CV[fold]

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

        for epoch in range(num_epochs):
            if epoch == warmup_epochs:
                warmup = False
                for param in model.coe_var_layer.parameters():
                    param.requires_grad = True

            model.train()
            train_loss = 0.0
            for batch in train_loader:
                inputs, labels = batch
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels, beta, a, warmup, b_weight)
                loss_val = criterion_val(outputs, labels, a, warmup, b_weight)
                loss.backward()
                if epoch >= warmup_epochs:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                train_loss += loss_val.item()
            train_loss /= len(train_loader)

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    outputs = model(inputs)
                    loss = criterion_val(outputs, targets, a, warmup, b_weight)
                    val_loss += loss.item()
            val_loss /= len(val_loader)

            if epoch > warmup_epochs_early_stopping:
                early_stopping(val_loss)
                if early_stopping.early_stop:
                    print(f"Early stopping at epoch {epoch + 1}")
                    break
        
        model_state_dict_10[fold].append(model.state_dict())
        val_loss_fold.append(val_loss)
        print(f"Fold {fold + 1}, Model {m + 1}, Validation loss: {val_loss}")
    
with open('Best_network/best_NN_NLL_cov_V2_10.pth','wb') as f:
    torch.save({'model_state':model_state_dict_10,
                'lr':lr,
                'num_neuron':best_model_dict['num_neuron'],
                'num_layer':best_model_dict['num_layer'],
                'batch_size':batch_size,
                'weight_decay':weight_decay,
                'warmup_epochs':warmup_epochs},f)


Early stopping at epoch 779
Fold 1, Model 1, Validation loss: 3.361151933670044
Early stopping at epoch 779
Fold 1, Model 2, Validation loss: 2.5899298191070557
Early stopping at epoch 779
Fold 1, Model 3, Validation loss: 2.2884302139282227
Early stopping at epoch 780
Fold 1, Model 4, Validation loss: 2.742144823074341
Early stopping at epoch 779
Fold 1, Model 5, Validation loss: 6.782578468322754
Early stopping at epoch 779
Fold 1, Model 6, Validation loss: 2.1262478828430176
Early stopping at epoch 779
Fold 1, Model 7, Validation loss: 3.5219531059265137
Early stopping at epoch 779
Fold 1, Model 8, Validation loss: 2.6193976402282715
Early stopping at epoch 790
Fold 1, Model 9, Validation loss: 2.7740678787231445
Early stopping at epoch 779
Fold 1, Model 10, Validation loss: 8.166481018066406
Early stopping at epoch 876
Fold 2, Model 1, Validation loss: 3.3756256103515625
Early stopping at epoch 1114
Fold 2, Model 2, Validation loss: 3.0240554809570312
Early stopping at epoch 779
Fo

In [24]:
b_train_pred_10_all = {}
b_test_in_pred_10_all = {}
b_test_out_pred_10_all = {}

for fold in range(10):
    b_train_pred_10 = []
    b_test_in_pred_10 = []
    b_test_out_pred_10 = []

    for m in range(M):
        model = Network_cof_variation(input_size, hidden_sizes, output_size)
        model.load_state_dict(model_state_dict_10[fold][m])

        model.eval()
        with torch.no_grad():
            b_train_pred_fold_m = model(X_train_PCA_tensor).numpy()
            b_test_in_pred_fold_m = model(X_test_in_PCA_tensor).numpy()
            b_test_out_pred_fold_m = model(X_test_out_PCA_tensor).numpy()

        b_train_pred_10.append(b_train_pred_fold_m)
        b_test_in_pred_10.append(b_test_in_pred_fold_m)
        b_test_out_pred_10.append(b_test_out_pred_fold_m)

    b_train_pred_10_all[fold] = np.array(b_train_pred_10)
    b_test_in_pred_10_all[fold] = np.array(b_test_in_pred_10)
    b_test_out_pred_10_all[fold] = np.array(b_test_out_pred_10)


In [ ]:

with open('Empirical_parameter_results/E2E_NNE_V2/b_train_pred_10_all.pkl','wb') as f:
    pickle.dump(b_train_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/b_test_in_pred_10_all.pkl','wb') as f:
    pickle.dump(b_test_in_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/b_test_out_pred_10_all.pkl','wb') as f:
    pickle.dump(b_test_out_pred_10_all,f)

In [ ]:
Q_train_pred_10_all = {}
Q_test_in_pred_10_all = {}
Q_test_out_pred_10_all = {}

std_train_pred_10_all = {}
std_test_in_pred_10_all = {}
std_test_out_pred_10_all = {}

Q_train_pred_10_ensemble = {}
Q_test_in_pred_10_ensemble = {}
Q_test_out_pred_10_ensemble = {}

std_train_pred_10_ensemble = {}
std_test_in_pred_10_ensemble = {}
std_test_out_pred_10_ensemble = {}

for fold in range(10):
    Q_train_hat_all_models_fold = np.array([[empirical_model(a, b1, b2, b3, N_train[i],b_weight)*100 for i, (b1, b2, b3, _) in enumerate(model_b_train_hat)] for model_b_train_hat in b_train_pred_10_all[fold]])
    Q_test_in_hat_all_models_fold = np.array([[empirical_model(a, b1, b2, b3, N_test_in[i],b_weight)*100 for i, (b1, b2, b3, _) in enumerate(model_b_test_in_hat)] for model_b_test_in_hat in b_test_in_pred_10_all[fold]])
    Q_test_out_hat_all_models_fold = np.array([[empirical_model(a, b1, b2, b3, N_test_out[i],b_weight)*100 for i, (b1, b2, b3, _) in enumerate(model_b_test_out_hat)] for model_b_test_out_hat in b_test_out_pred_10_all[fold]])

    coe_var_train_all_models_fold = np.array([np.repeat(model_b_train_hat[:, 3][:, np.newaxis], Q_train.shape[1], axis=1) for model_b_train_hat in b_train_pred_10_all[fold]])
    coe_var_test_in_all_models_fold = np.array([np.repeat(model_b_test_in_hat[:, 3][:, np.newaxis], Q_test_in.shape[1], axis=1) for model_b_test_in_hat in b_test_in_pred_10_all[fold]])
    coe_var_test_out_all_models_fold = np.array([np.repeat(model_b_test_out_hat[:, 3][:, np.newaxis], Q_test_out.shape[1], axis=1) for model_b_test_out_hat in b_test_out_pred_10_all[fold]])

    # Calculate the variance for each individual model
    std_train_all_models_fold = coe_var_train_all_models_fold * (100 - Q_train_hat_all_models_fold)
    std_test_in_all_models_fold = coe_var_test_in_all_models_fold * (100 - Q_test_in_hat_all_models_fold)
    std_test_out_all_models_fold = coe_var_test_out_all_models_fold * (100 - Q_test_out_hat_all_models_fold)

    var_train_all_models_fold = std_train_all_models_fold ** 2
    var_test_in_all_models_fold = std_test_in_all_models_fold ** 2
    var_test_out_all_models_fold = std_test_out_all_models_fold ** 2

    Q_train_hat_combined_fold = np.mean(Q_train_hat_all_models_fold, axis=0)
    Q_test_in_hat_combined_fold = np.mean(Q_test_in_hat_all_models_fold, axis=0)
    Q_test_out_hat_combined_fold = np.mean(Q_test_out_hat_all_models_fold, axis=0)

    # Calculate the variance of the combined predictions
    var_train_combined_fold = np.mean(var_train_all_models_fold + Q_train_hat_all_models_fold**2 - Q_train_hat_combined_fold**2, axis=0)
    var_test_in_combined_fold = np.mean(var_test_in_all_models_fold + Q_test_in_hat_all_models_fold**2 - Q_test_in_hat_combined_fold**2, axis=0)
    var_test_out_combined_fold = np.mean(var_test_out_all_models_fold + Q_test_out_hat_all_models_fold**2 - Q_test_out_hat_combined_fold**2, axis=0)

    std_train_combined_fold = np.sqrt(var_train_combined_fold)
    std_test_in_combined_fold = np.sqrt(var_test_in_combined_fold)
    std_test_out_combined_fold = np.sqrt(var_test_out_combined_fold)

    Q_train_pred_10_all[fold] = Q_train_hat_all_models_fold
    Q_test_in_pred_10_all[fold] = Q_test_in_hat_all_models_fold
    Q_test_out_pred_10_all[fold] = Q_test_out_hat_all_models_fold

    std_train_pred_10_all[fold] = std_train_all_models_fold
    std_test_in_pred_10_all[fold] = std_test_in_all_models_fold
    std_test_out_pred_10_all[fold] = std_test_out_all_models_fold

    Q_train_pred_10_ensemble[fold] = Q_train_hat_combined_fold
    Q_test_in_pred_10_ensemble[fold] = Q_test_in_hat_combined_fold
    Q_test_out_pred_10_ensemble[fold] = Q_test_out_hat_combined_fold

    std_train_pred_10_ensemble[fold] = std_train_combined_fold
    std_test_in_pred_10_ensemble[fold] = std_test_in_combined_fold
    std_test_out_pred_10_ensemble[fold] = std_test_out_combined_fold

with open('Empirical_parameter_results/E2E_NNE_V2/Q_train_pred_10_all.pkl','wb') as f:
    pickle.dump(Q_train_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_in_pred_10_all.pkl','wb') as f:
    pickle.dump(Q_test_in_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_out_pred_10_all.pkl','wb') as f:
    pickle.dump(Q_test_out_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_train_pred_10_all.pkl','wb') as f:
    pickle.dump(std_train_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_in_pred_10_all.pkl','wb') as f:
    pickle.dump(std_test_in_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_out_pred_10_all.pkl','wb') as f:
    pickle.dump(std_test_out_pred_10_all,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_train_pred_10_ensemble.pkl','wb') as f:
    pickle.dump(Q_train_pred_10_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_in_pred_10_ensemble.pkl','wb') as f:
    pickle.dump(Q_test_in_pred_10_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/Q_test_out_pred_10_ensemble.pkl','wb') as f:
    pickle.dump(Q_test_out_pred_10_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_train_pred_10_ensemble.pkl','wb') as f:
    pickle.dump(std_train_pred_10_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_in_pred_10_ensemble.pkl','wb') as f:
    pickle.dump(std_test_in_pred_10_ensemble,f)

with open('Empirical_parameter_results/E2E_NNE_V2/std_test_out_pred_10_ensemble.pkl','wb') as f:
    pickle.dump(std_test_out_pred_10_ensemble,f)

In [ ]:
MAE_training = []
MAE_test_in = []
MAE_test_out = []

RMSE_training = []
RMSE_test_in = []
RMSE_test_out = []

for fold in range(10):
    MAE_training_fold = []
    MAE_test_in_fold = []
    MAE_test_out_fold = []

    RMSE_training_fold = []
    RMSE_test_in_fold = []
    RMSE_test_out_fold = []

    for i in range(num_training_cells):
        MAE_training_fold.append(mean_absolute_error(Q_train[i]*100,Q_train_pred_10_ensemble[fold][i]))
        RMSE_training_fold.append(root_mean_squared_error(Q_train[i]*100,Q_train_pred_10_ensemble[fold][i]))

    for i in range(num_test_in_cells):
        MAE_test_in_fold.append(mean_absolute_error(Q_test_in[i]*100,Q_test_in_pred_10_ensemble[fold][i]))
        RMSE_test_in_fold.append(root_mean_squared_error(Q_test_in[i]*100,Q_test_in_pred_10_ensemble[fold][i]))

    for i in range(num_test_out_cells):
        MAE_test_out_fold.append(mean_absolute_error(Q_test_out[i]*100,Q_test_out_pred_10_ensemble[fold][i]))
        RMSE_test_out_fold.append(root_mean_squared_error(Q_test_out[i]*100,Q_test_out_pred_10_ensemble[fold][i]))

    MAE_training.append(np.mean(MAE_training_fold))
    MAE_test_in.append(np.mean(MAE_test_in_fold))
    MAE_test_out.append(np.mean(MAE_test_out_fold))

    RMSE_training.append(np.mean(RMSE_training_fold))
    RMSE_test_in.append(np.mean(RMSE_test_in_fold))
    RMSE_test_out.append(np.mean(RMSE_test_out_fold))

result_dict = {'MAE_training':MAE_training,
                'MAE_test_in':MAE_test_in,
                'MAE_test_out':MAE_test_out,
                'RMSE_training':RMSE_training,
                'RMSE_test_in':RMSE_test_in,
                'RMSE_test_out':RMSE_test_out}

with open('Empirical_parameter_results/E2E_NNE_V2/result_dict_10.pkl','wb') as f:
    pickle.dump(result_dict,f)

mean_MAE_training = np.mean(MAE_training)
mean_MAE_test_in = np.mean(MAE_test_in)
mean_MAE_test_out = np.mean(MAE_test_out)

mean_RMSE_training = np.mean(RMSE_training)
mean_RMSE_test_in = np.mean(RMSE_test_in)
mean_RMSE_test_out = np.mean(RMSE_test_out)

print(f"Mean MAE training: {mean_MAE_training:.2f}")
print(f"Mean MAE test in: {mean_MAE_test_in:.2f}")
print(f"Mean MAE test out: {mean_MAE_test_out:.2f}")

print(f"Mean RMSE training: {mean_RMSE_training:.2f}")
print(f"Mean RMSE test in: {mean_RMSE_test_in:.2f}")
print(f"Mean RMSE test out: {mean_RMSE_test_out:.2f}")

Mean MAE training: 1.73
Mean MAE test in: 2.05
Mean MAE test out: 7.14
Mean RMSE training: 2.11
Mean RMSE test in: 2.47
Mean RMSE test out: 8.82


In [27]:
print(f"MAE training: {MAE_training}")
print(f"MAE test in: {MAE_test_in}")
print(f"MAE test out: {MAE_test_out}")

MAE training: [1.784060336859118, 1.8386336529389424, 1.632692972460559, 1.626906289616892, 1.7307085151023, 1.7469161020868753, 1.6925760692281258, 1.703419231528423, 1.7322012733345071, 1.7724808515131016]
MAE test in: [2.2784796480191947, 2.1662178337546334, 1.9765103565248399, 1.9790225691551302, 1.7690244412170895, 2.2759907825362307, 1.7942676720690807, 1.9604396249642178, 2.277860275059515, 2.043930906690721]
MAE test out: [8.528357093958984, 7.5104533379726846, 8.198342718556477, 6.460058254895792, 3.9085260366471757, 8.791309163180296, 5.630577073762923, 6.4752537960282845, 8.233896754263895, 7.631773181479316]
